In [1]:
import seaborn as sns
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
import numpy as np
import bz2
import json
from urllib.parse import urlparse
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import os
import glob

In [12]:
#creation d'un nouveau fichier avec une nouvelle colonne pour le domaine (cf colab)
from tld import get_tld

def get_domain(url):
    res = get_tld(url, as_object=True)
    return res.tld

In [13]:
#creation d'un fichier avec des colonnes en plus: le domaine et le nom du site internet du lien
path_to_file = 'quotes-2017.json.bz2' 
path_to_out = '2017a.json.bz2'

with bz2.open(path_to_file, 'rb') as s_file:
    with bz2.open(path_to_out, 'wb') as d_file:
        for instance in s_file:
            instance = json.loads(instance) # loading a sample
            urls = instance['urls'] # extracting list of links
            domains = []
            website = []
            for url in urls:
                tld = get_domain(url)
                domains.append(tld)
                net = urlparse(url)
                neto=net.netloc
                website.append(neto)
            instance['domains'] = domains # updating the sample with domain name
            instance['website'] = website
            d_file.write((json.dumps(instance)+'\n').encode('utf-8')) # writing in the new file

In [ ]:
#creation du dataframe à partir des 5 années avec les citations contenant un mot précis, ici immigrat
path = os.getcwd()
json_files = glob.glob(os.path.join(path, "*.json.bz2"))
lista='quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences','probas', 'urls', 'phase', 'domains', 'website'
df2=pd.DataFrame(columns=lista) 
def process_chunk(chunk,df2):
        immi=pd.DataFrame()
        immi=chunk[chunk['quotation'].str.contains("immigrat")]
        df2 = df2.append(immi)
        print(df2.shape)
        return df2
for f in json_files:      
    for chunk in pd.read_json(f, lines=True, compression='bz2', chunksize=250000, encoding='utf-8'):
        df2=process_chunk(chunk,df2)
    print('a')

In [ ]:
#creation du dataframe à partir des 5 années avec les citations d'un site en particulier
path = os.getcwd()
json_files = glob.glob(os.path.join(path, "*.json.bz2"))
lista='quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences','probas', 'urls', 'phase', 'domains', 'website'
df3=pd.DataFrame(columns=lista) 
def process_chunk(chunk,df3):
        chunk['website']=chunk['website'].astype('str')
        journal=pd.DataFrame()
        journal=chunk[chunk['website'].str.contains("www.foxnews.com")]
        df3 = df3.append(journal)
        #print(df3.shape)
        return df2
for f in json_files:      
    for chunk in pd.read_json(f, lines=True, compression='bz2', chunksize=25000, encoding='utf-8'):
        df3=process_chunk(chunk,df3)
    print('a')

In [ ]:
df3.head(10)

In [10]:
df3.shape

(3456, 11)

In [40]:
#ici on voit que la colonne ajouter fonctionne
lis=df2['website'].value_counts().to_frame()
lis.head(5)

,website
['www.msn.com'],1202
['express.co.uk'],1184
['www.cheatsheet.com'],925
['www.bostonglobe.com'],769
['www.foxnews.com'],763
['timesofindia.indiatimes.com'],745
['www.dailystar.co.uk'],742
['www.telegraph.co.uk'],741
['indianexpress.com'],741
['www.thesun.co.uk'],730


In [15]:
lis2=df3['speaker'].value_counts().to_frame()
lis2.head(5)

,speaker
None,84128
President Donald Trump,1258
Bernie Sanders,595
Joe Biden,581
President Trump,546
...,...
Kirstjen Nielsen,1
Jennifer Anderson,1
alan tongue,1
Wes Welker,1


In [3]:
#changing the website column type to string
df2['website']=df2['website'].astype('str')

In [52]:
#creation d'un dataframe pour quelques journaux
fox=df2[df2['website'].str.contains("www.foxnews.com")]
ny=df2[df2['website'].str.contains("www.nytimes.com")]
brei=df2[df2['website'].str.contains("www.breitbart.com")]
cnn=df2[df2['website'].str.contains("cnn.com")]
guard=df2[df2['website'].str.contains("www.theguardian.com")]
slate=df2[df2['website'].str.contains("slate.com")]
buzz=df2[df2['website'].str.contains("buzzfeed.com")]

In [53]:
fox_quotes=fox['quotation']
ny_quotes=ny['quotation']
brei_quotes=brei['quotation']
cnn_quotes=cnn['quotation']
guard_quotes=guard['quotation']
slate_quotes=slate['quotation']
buzz_quotes=buzz['quotation']

In [8]:
sia = SentimentIntensityAnalyzer()

In [8]:
fox.shape
ny.shape
brei.shape
cnn.shape
guard.shape
slate.shape
buzz.shape

(1787, 11)

In [16]:
#analyse de sentiments positif et negatif pour les journaux précédemment choisis
pos=0
neg=0
average_pos=0
average_neg=0
for quotation in fox_quotes:
    result=sia.polarity_scores(quotation)
    pos=pos+result["pos"]
    neg=neg+result["neg"]
average_pos=pos/1787
average_neg=neg/1787
print(average_pos,average_neg)

0.11182372691661988 0.07998601007274761


In [12]:
df3_quotes=df3['quotation']
df3size=df3_quotes.shape(1)

In [13]:
pos=0
neg=0
average_pos=0
average_neg=0
for quotation in df3_quotes:
    result=sia.polarity_scores(quotation)
    pos=pos+result["pos"]
    neg=neg+result["neg"]
average_pos=pos/df3size
average_neg=neg/df3size
print(average_pos,average_neg)

0.09833275462962988 0.08480266203703696
